<a href="https://colab.research.google.com/github/Arpit-Babbar/arpit_practise/blob/main/python/operator_as_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We are given a matrix operator(i.e., fourth order tensor) 
$$
T_A:\mathbb{M}^{n+1} \to \mathbb{M}^{n+1}
$$
defined as 
$$
T_A(B) = AB
$$
where $A,B$ are $n+1 \times n+1$ matrices.

We wish to see $T_A$ as an $(n+1)^2 \times (n+1)^2$ matrix acting on vectors of size $(n+1)^2$ so that Python can compute its eigenvalues. 

We'd create a basis representation map $\Psi : \mathbb{M}^{n+1} \to \mathbb{R}^{(n+1)^2}$ so that corresponding to our operator $T_A$, we have a matrix $\mathbf{A}_{(n+1)^2\times (n+1)^2}$ so that, for a matrix $B$, defining $\mathbf{b}_{(n+1)^2 \times 1} := \Psi(B)$, we have 
$$ \Psi^{-1}[\mathbf{A} \mathbf{b}] = T_A(B) $$


We order the standard basis $$\{E_{ij}\}_{0 \le i,j \le n}$$ as 
$$\{E_{00},E_{01},\dots,E_{0n},E_{10},E_{11},\dots,E_{nn} \}.$$
That is, we choose the basis to be $\{e_k\}_{k:0}^{(n+1)^2}$ where
$$e_k = E_{\left[\frac{k}{n+1}\right],k-\left[\frac{k}{n+1}\right](n+1)} $$
where $[x]$ gives the greatest integer less than $x$.

In [1]:
import numpy as np
from numpy import floor,sqrt

Let's use the formula to write a matrix as a vector and then back(For going back, we need to note that $E_{ij}=e_{i(n+1)+j}$).

In [2]:
def mat2vec(B): #Converts a matrix to vector
    n= np.shape(B)[0]-1
    b = np.zeros(((n+1)**2,1))
    for k in range(0,(n+1)**2):
        i,j =floor(k/(n+1)),k-np.floor(k/(n+1))*(n+1)
        i,j=int(i),int(j)
        b[k,0] = B[i,j]
    return np.array(b)
def vec2mat(b): #sending vector representation back to matrix
    N=np.shape(b)[0]
    assert (sqrt(N).is_integer()),'Only perfect square \
            sized 1-d arrays can be represented as matrices'
    n=sqrt(N)-1
    n=int(n)
    B = np.zeros((n+1,n+1))
    for i in range(0,n+1):
        for j in range(0,n+1):
            B[i,j]= b[i*(n+1)+j,0]
    return B

Let's test that the functions work

In [3]:
B = np.array([ \
              [1,2,3],\
              [4,5,6],\
              [7,8,9]\
             ])
b = mat2vec(B)
B = vec2mat(b)
print(b)
print(B)

[[1.]
 [2.]
 [3.]
 [4.]
 [5.]
 [6.]
 [7.]
 [8.]
 [9.]]
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


So, we can represent a matrix as an array and then go back to our matrix.
We will now use it to represent $T_A$ as a matrix. By definition of the matrix representative $\mathbf{A}$, $$T_A(e_k)=\sum_{l:0}^{(n+1)^2} \mathbf{A}_{lk} e_l \quad \text{ for } 0\le k \le (n+1)^2$$
We can solve this equation using `numpy`, but that will be too inefficient. Let's do it explicitly.

Since $e_k = E_{\left[\frac{k}{n+1}\right],k-\left[\frac{k}{n+1}\right](n+1)}$, $T_A(e_k) = AE_{\left[\frac{k}{n+1}\right],k-\left[\frac{k}{n+1}\right](n+1)}$. That is,

Letting $A_{pq}$ denote coefficients of $A$, 
$$T_A(e_k) = \sum_{i:0}^n A_{i,\left[\frac{k}{n+1}\right]}E_{i,k-\left[\frac{k}{n+1}\right](n+1)}$$
That is,
$$T_A(e_k) = \sum_{i:0}^n A_{i,\left[\frac{k}{n+1}\right]}e_{(n+1)i+k-\left[\frac{k}{n+1}\right](n+1)}$$
Thus $$
\mathbf{A}_{lk} = 
\begin{cases}
A_{i_0,\left[\frac{k}{n+1}\right]}\quad \text{ if } l=(n+1)i_0+k-\left[\frac{k}{n+1}\right](n+1) \text{ for some }i_0 \\
0, \quad \text{ otherwise}
\end{cases}
$$



So, we'd look over $k,l$ and define
$$i:=\frac{l-k+\left[\frac{k}{n+1}\right](n+1)}{n+1}.$$
If it's an integer, we will put the respective value in the matrix. A more efficient way would be to simply loop over $i$ and determine the corresponding $k,l$. I don't know how to do it though. Maybe, loop over $k,i$ instead!? Loop $k$ over $0,...,n^2$ and $i$ over $0,...,n$. That is, the expression of $T_A(e_k)$ give all the non-zero entries and their positions. So, just loop over the parameters that give you the position and put it in.

Let's create the function that writes our operator as a matrix, we will denote the $(n+1)\times(n+1)$ matrix by `A` and its $(n+1)^2\times (n+1)^2$ representation by `AA`.

In [4]:
def tensor2mat(A):
    n=np.shape(A)[0]-1
    AA = np.zeros(((n+1)**2,(n+1)**2))
    for k in range(0,(n+1)**2):
        for l in range(0,(n+1)**2):
            i_num = l-k+floor(k/(n+1))*(n+1)
            i_den = n+1
            i     = i_num/i_den
            if i.is_integer():
                i = int(i)
                j = int(floor(k/(n+1)))
                AA[l,k]=A[i,j]
    return AA

We test that it works by using random matrices

In [10]:
A = np.random.rand(3,3)
B = np.random.rand(3,3)
AA = tensor2mat(A)
b = mat2vec(B)

In [11]:
print(A)
print(AA)

[[0.76525026 0.54337493 0.83179074]
 [0.49710136 0.75726695 0.4561702 ]
 [0.28351764 0.38436602 0.24205699]]
[[0.76525026 0.         0.         0.54337493 0.         0.
  0.83179074 0.         0.        ]
 [0.         0.76525026 0.         0.         0.54337493 0.
  0.         0.83179074 0.        ]
 [0.         0.         0.76525026 0.         0.         0.54337493
  0.         0.         0.83179074]
 [0.49710136 0.         0.         0.75726695 0.         0.
  0.4561702  0.         0.        ]
 [0.         0.49710136 0.         0.         0.75726695 0.
  0.         0.4561702  0.        ]
 [0.         0.         0.49710136 0.         0.         0.75726695
  0.         0.         0.4561702 ]
 [0.28351764 0.         0.         0.38436602 0.         0.
  0.24205699 0.         0.        ]
 [0.         0.28351764 0.         0.         0.38436602 0.
  0.         0.24205699 0.        ]
 [0.         0.         0.28351764 0.         0.         0.38436602
  0.         0.         0.24205699]]


In [ ]:
AA@b

array([[0.69446933],
       [1.00523709],
       [0.73041863],
       [1.10646946],
       [1.56798594],
       [1.32589614],
       [0.21080779],
       [0.5023695 ],
       [0.3631381 ]])

In [ ]:
vec2mat(AA@b)

array([[0.69446933, 1.00523709, 0.73041863],
       [1.10646946, 1.56798594, 1.32589614],
       [0.21080779, 0.5023695 , 0.3631381 ]])

In [ ]:
A@B

array([[0.69446933, 1.00523709, 0.73041863],
       [1.10646946, 1.56798594, 1.32589614],
       [0.21080779, 0.5023695 , 0.3631381 ]])

Since both the ways give us the same result, our representation is correct.

Now, we shall do the same thing for an operator of the form $U_A(B) = BA$. That is, we will find $\mathbf{A}_{(n+1)^2\times (n+1)^2}$ so that $$U_A(B) = \mathbf{A}B$$ As before, 
$$U_A(e_k)=\sum_{l:0}^{(n+1)^2} \mathbf{A}_{lk} e_l \quad \text{ for } 0\le k \le (n+1)^2$$
Letting $A_{pq}$ denote coefficients of $A$,  for $0\le k \le (n+1)^2$, $U_A(e_k) = e_kA = E_{\left[\frac{k}{n+1}\right],k-\left[\frac{k}{n+1}\right](n+1)}A$ Noting that $$E_{pq}A = \sum_{j:0}^n A_{qj}E_{pj}, $$
we have
$$E_{\left[\frac{k}{n+1}\right],k-\left[\frac{k}{n+1}\right](n+1)}A = \sum_{j:0}^n A_{k-\left[\frac{k}{n+1}\right](n+1),j}E_{\left[\frac{k}{n+1}\right],j},$$
That is,
$$U_A(e_k) = \sum_{j:0}^n A_{k-\left[\frac{k}{n+1}\right](n+1),j}e_{\left[\frac{k}{n+1}\right](n+1)+j}$$
Thus $$
\mathbf{A}_{lk} = 
\begin{cases}
A_{k-\left[\frac{k}{n+1}\right](n+1),j}\quad \text{ if } l= \left[\frac{k}{n+1}\right](n+1)+j \text{ for some $j$ }\\
0, \quad \text{ otherwise}
\end{cases}
$$

So, we loop over $j=0,...,n$ and $k=0,\dots,n^2$, get the non-zero entries $(p,q)$ and put them in their respective position $(l,k)$.

In [12]:
def ltensor2mat(A):
    n=np.shape(A)[0]-1
    AA = np.zeros(((n+1)**2,(n+1)**2))
    for k in range(0,(n+1)**2):
        for j in range(0,n+1):
            p = k-floor(k/(n+1))*(n+1)
            q = j
            l = floor(k/(n+1))*(n+1)+j
            p,q,l=int(p),int(q),int(l)
            AA[l,k]=A[p,q]
    return AA

In [15]:
A = np.random.rand(3,3)
B = np.random.rand(3,3)
AA = ltensor2mat(A)
b = mat2vec(B)

In [16]:
AA@b

array([[1.30885196],
       [1.53375694],
       [0.5260633 ],
       [1.18418272],
       [1.36080432],
       [0.51086942],
       [0.82811321],
       [1.01260441],
       [0.27217159]])

In [17]:
vec2mat(AA@b)

array([[1.30885196, 1.53375694, 0.5260633 ],
       [1.18418272, 1.36080432, 0.51086942],
       [0.82811321, 1.01260441, 0.27217159]])

In [18]:
B@A

array([[1.30885196, 1.53375694, 0.5260633 ],
       [1.18418272, 1.36080432, 0.51086942],
       [0.82811321, 1.01260441, 0.27217159]])